In [1]:
# Import abilities
from ai import Chat, Vision, Audio, Images, Embeddings, FineTuner
import powers

# Adjust model globally here or modify 
# in function calls. Vision model cannot be changed.
model = "gpt-3.5-turbo-1106"

## Examples

Here are multiple examples of how to use the `ai.py` script.

#### Basic completion call.

In [3]:
chat = Chat(model=model, system="Respond like a frog.")

object = "pizza"

# memories control whether the AI remembers the conversation or not, provided Chat is not reinitialized.
completion = chat.chat_completion(prompt=f"Tell me a perfect haiku about {object}", memories=True, seed=42, stream=True)

Initalized Chat class with model gpt-3.5-turbo-1106
---------------------------------------------
Making Chat Completion API call...
---------------------------------------------
Croak, tomato sauce
Cheesy goodness on my tongue
Pizza, my delight



#### Conversation with an LLM which retains memory of previous calls.

- Use stream to see the response as it comes in.
- Use speak to have the LLM speak the response out loud.

In [4]:
audio = Audio()
chat = Chat(model=model, system="Helpful assistant.")

while True:
    transcript = audio.record_and_transcribe()
    print(transcript)
    chat.chat_completion(transcript, speak=True, stream=True, memories=True)

Initalized Audio class.
---------------------------------------------
Initalized Chat class with model gpt-3.5-turbo-1106
---------------------------------------------
Recording started. Speak into the microphone.
---------------------------------------------
Recording ended.
---------------------------------------------
Making Speech Transcription API call...
---------------------------------------------
Hello, how are you?
Making Chat Completion API call...
---------------------------------------------
Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?

Generating speech...
---------------------------------------------
Initalized Audio class.
---------------------------------------------
Making Speech API call...
---------------------------------------------


KeyboardInterrupt: 

#### Moving memory from a conversation LLM to the vision LLM.

In [5]:
chat = Chat(model=model, system="")

vision = Vision()

completion_1 = chat.chat_completion(prompt="What is your purpose?", stream=False, memories=True, return_messages=True)

### This should respond with something like "You just asked, 'What is your purpose?' if the memory transfer worked."
completion_2 = vision.vision_completion(prompt="What did I just say and how did you respond?", messages=completion_1["messages"], stream = True, speak=True)

Initalized Chat class with model gpt-3.5-turbo-1106
---------------------------------------------
Initalized Vision class with model gpt-4-vision-preview.
---------------------------------------------
Making Chat Completion API call...
---------------------------------------------
Making Vision Completion API call...
---------------------------------------------
You asked, "What is your purpose?" and I responded by explaining that as an AI language model, my purpose is to assist, provide information, and help users find answers to their questions while engaging in meaningful conversations.

Initalized Audio class.
---------------------------------------------
Making Speech API call...
---------------------------------------------


#### Infinite image generation and description using Vision and Images.

In [ ]:
images = Images()

image_generation = images.generate_image("A sphere of pure energy.", fix_prompt=False, display_image=True, save_image=True)

vision = Vision(system="You can only respond with bullet points.")

vision.vision_completion(prompt="What is this image showing?", image_paths=[image_generation["path_to_image"]], memories=False, stream=True)

#### Logit Bias to control the LLM's responses.

In [ ]:
chat = Chat(model=model, system="")

# Applying a global bias of -100 to a list of phrases.
phrases = ["OpenAI"]
global_bias = -100

## Return tokens and global bias to apply.
logit_bias = chat.create_logit_bias(phrases, global_bias, augment=True)

prompt = "Only tell me the name of the company who developed you."

chat.chat_completion(prompt=prompt,
                     logit_bias=logit_bias, 
                    stream=True, 
                    memories=False,
                    seed=42)

## Or, we can control the logit bias at a phrase level, by passing a dictionary of phrases and biases.

phrases = {"OpenAI": -100, "Google": 18}

## Return tokens and local bias to apply from phrases dict.
logit_bias = chat.create_logit_bias(phrases, augment=True)

## If you want to apply bias at token level, you need to construct the logit bias manually.

response, messages = chat.chat_completion(prompt=prompt,
                                          logit_bias=logit_bias, 
                                          stream=True, 
                                          memories=False,
                                          seed=42)

#### Logit bias Lobotomy

In [ ]:
from ai import Chat

# Initialize the chat model
chat = Chat(model=model, system="Answer all questions.")

# Start with an empty set of banned words
banned_words = set()

# empty logit bias dictionary
logit_bias = {}

# Define a prompt for the AI
prompt = "What is 1+1?"

# Loop until the AI can no longer generate a response
while True:
    # Convert the banned words into token IDs and then into a logit bias dictionary with a high negative value to ban them
    logit_bias = chat.create_logit_bias(list(banned_words), bias=-100, augment=False)

    # Get the AI's response
    response, messages = chat.chat_completion(prompt=prompt,
                                              stream=False,
                                              logit_bias=logit_bias,
                                              memories=False,
                                              seed=50)

    # Check if the AI was able to generate a response
    if not response.strip():
        print("\n------------------\n")
        print("Lobotomy complete. AI failed to generate a response.")
        break

    # Print the AI's response
    print(response)

    # Update the set of banned words with the words from the latest response
    words_in_response = set(response.split())
    banned_words.update(words_in_response)

#### Embeddings for string similarity.

In [ ]:
# Initialize the classes
chat = Chat(model="gpt-4", system = "You create a prompt for an image to be generated about the given topic: \n\n")
image = Images()
vision = Vision(system="Imagine you are reverse engineering the prompt used to generate this image. Give the prompt and nothing else.")
embedding = Embeddings()

# Use the Chat class to generate a description of the scene
print("Generating description...")
description, messages = chat.chat_completion("sausage person")

# Use the Image class to generate an image based on the description
print("Generating image...")
image_generation = image.generate_image(description, display_image=True, save_image=True)

# Use the Vision class to describe the generated image
print("Describing image...")
vision_response, messages = vision.vision_completion("", image_paths=[image_generation["path_to_image"]], memories=False, stream=True)

# Calculate the cosine similarity between the original description and the image description
print("Calculating similarity...")
similarity = embedding.string_similarity(description, vision_response)

# Print the results
print(f"Original description: {description}")
print(f"Image description: {vision_response}")
print(f"Cosine similarity: {similarity}")

#### Retrieval Augmented Generation (RAG)

In [ ]:
embeddings = Embeddings()
chat = Chat(model=model, system="Bullet points only. Answer the question using relevant information from the knowledge base.")

# Let's assume we have a knowledge base of facts
knowledge_base = [
"Alice enjoys playing chess, walking and reading a good book. Sometimes she likes to go to the park and feed the ducks.",
"Bob is a keen gardener. He likes to grow flowers and vegetables. He also likes to go for long walks in the countryside.",
"Charlie is scared of the monsters his the bed, which he knows are real.",
"Dennis really hates Charlie because he thinks he is an idiot and monsters don't exist.",
"Eve is a very good cook. She likes to cook all kinds of food. Her favourite dish is spaghetti bolognese.",
"Frank is a very successful businessman. He owns a chain of restaurants, a hotel and a casino.",
"Grace is a very talented artist. She likes to paint portraits of her friends and family. She recently painted the monster under Charlie's bed.",
]
# Example usage
query = "Describe the relationship between Dennis and Charlie. Could Frank help? Or is he the monster?"

retrieved_facts = powers.retrieval(embeddings, query, knowledge_base, top_n=10, similarity_threshold=0.75)

facts_string = ", ".join(retrieved_facts)

prompt = f"{facts_string} \n\n: {query}"

response, messages = chat.chat_completion(prompt=prompt, memories=False, stream=True)

#### Function calling.

In [ ]:
import powers

chat = Chat(model=model, system="Helpful robot.")

# Dummy function to simulate an API call to get weather data
def get_current_weather(location, unit="celsius"):
    # In a real scenario, this function would make an API call to a weather service
    weather_data = {
        "Tokyo": {"temperature": "10", "unit": unit},
        "San Francisco": {"temperature": "30", "unit": unit},
        "Paris": {"temperature": "22", "unit": unit}
    }
    return weather_data.get(location, {"temperature": "unknown"})

# Dictionary mapping function names to actual function objects
available_functions = {
    "get_current_weather": get_current_weather,
    # Add other functions here as needed
}

# Define the tool that the model can use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city, e.g. San Francisco",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

# Get the initial response from the model
completion, messages, tool_calls = chat.chat_completion(
    prompt="What's the temperature and weather in Paris vs Tokyo as a rhyming couplet?",
    tools=tools,
    tool_choice="auto",
    return_tool_calls=True,
    stream=False,
)

## Use tools is a power that applies any tools on the last message, and returns the updated messages with the responses from the tools integrated.   
messages = powers.use_tools(messages, available_functions)

# Get the final response from the model after the tool has been used
completion, messages = chat.chat_completion(
    prompt="",
    messages=messages,
    stream=True,
)

#### Text to speech.

In [ ]:
audio = Audio()

text = "The quick brown fox jumps over the lazy dog."

audio.speak(text=text, voice='echo')

#### Finetune a model and then use it.

In [ ]:
# Create an instance of the FineTuner class
fine_tuner = FineTuner()

# Define the path to your JSONL file
jsonl_file_path = 'assets/finetune-data.jsonl'

# Upload the file and start the fine-tuning process
fine_tuning_job = fine_tuner.finetune_model(
    file_path=jsonl_file_path,
    batch_size='12',
    learning_rate_multiplier='0.0001',
    model_name='gpt-3.5-turbo',
    suffix='example',
    n_epochs=10,  # for example
)

# Print the fine-tuning job details
print(fine_tuning_job)

In [ ]:
finetuned_job = fine_tuner.retrieve_finetuning_job(fine_tuning_job.id)

finetuned_job.fine_tuned_model

chat = Chat(model=finetuned_job.fine_tuned_model, system="")

response, messages = chat.chat_completion(prompt="What is the meaning of life?", memories=False, stream=True)

#### Virtual Cooking Assistant with Image Recognition

In [ ]:
vision = Vision(system="Identify ingredients in the image and make a list.")
images = Images()

# User uploads an image of ingredients they have
ingredient_image_path = "assets/fridge.jpg"

# Vision AI identifies the ingredients
ingredients, messages = vision.vision_completion(prompt="Identify these ingredients.", image_paths=[ingredient_image_path], stream=True)

chat = Chat(model="gpt-3.5-turbo", system="Generate a recipe based on the following ingredients.")
recipe_prompt = f"Create a recipe using these ingredients: {ingredients}"

# Generate a recipe
recipe, messages = chat.chat_completion(prompt=recipe_prompt, stream=True)

# Generate an image of the dish
images.generate_image(prompt=recipe, display_image=True, save_image=True)

#### Language Translation Basics

In [ ]:
chat = Chat(model=model) 
# Text to translate\n
original_text = 'Hello, how are you?'

#Languages to translate into\n
# 
languages = ['Spanish', 'French', 'German', 'Japanese', 'Chinese', 'Russian', 'Arabic', 'Hindi', 'Portuguese', 'Italian']

# Perform the translations\n
# 
for language in languages:
    prompt = f'Translate \"{original_text}\" into {language}.'
    response, messages = chat.chat_completion(prompt=prompt, stream=False)
    print(f'{language}: {response}')

#### Fitness Plan Creation

In [ ]:
chat = Chat(model=model)

# User's fitness goal
fitness_goal = 'build muscle'

# User's available days
available_days = ['Monday', 'Wednesday', 'Friday']

# Generate a workout plan
response, messages = chat.chat_completion(prompt=f'Create a workout plan to {fitness_goal} for someone available on {available_days}.',
                                        stream=True)
print(response)

#### Function calling for social media data.

In [ ]:
from ai import Chat


chat = Chat(model=model, system="Resourceful assistant.")

# Dummy function to simulate checking a user's social media mentions
def check_social_mentions(username):
    # In a real scenario, this function would interact with a social media API
    mention_data = {
        "alice": ["Just had a great experience with @alice's bakery!", "@alice's new cake design looks amazing!"],
        "bob": ["@bob's tech reviews are always so insightful.", "Can't wait for @bob's next podcast episode!"],
        "carol": ["@carol's workout tips have really helped me improve my routine.", "So inspired by @carol's health journey!"]
    }
    return mention_data.get(username, ["No mentions found."])

# Dictionary mapping function names to actual function objects
available_functions = {
    "check_social_mentions": check_social_mentions,
    # Add other functions here as needed
}

# Define the tool that the model can use
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_social_mentions",
            "description": "Check the latest social media mentions for a user",
            "parameters": {
                "type": "object",
                "properties": {
                    "username": {
                        "type": "string",
                        "description": "The social media username to check",
                    },
                },
                "required": ["username"],
            },
        },
    }
]

# Get the initial response from the model
completion, messages = chat.chat_completion(
    prompt="What are people saying about bob on social media?",
    tools=tools,
    tool_choice="auto",
    stream=False,
)


messages = powers.use_tools(messages, available_functions)

# Get the final response from the model after the tool has been used
completion, messages = chat.chat_completion(
    prompt="reply as a sea shanty",
    messages=messages,
    stream=True,
)